# Notebook 3: Local Outlier Factor (LOF) - Anomalias Locais

## Objetivo
Demonstrar como o LOF pode encontrar anomalias que métodos globais (como KNN simples ou estatística básica) falhariam em detectar.

## O Conceito
O LOF compara a densidade local de um ponto com a densidade local de seus vizinhos.
- Se um ponto tem uma densidade muito menor que seus vizinhos, ele é um outlier.
- Isso é útil quando temos dados com 'clusters' de diferentes densidades (ex: transações de departamentos diferentes com padrões diferentes).

In [ ]:
!pip install -q pyod pandas matplotlib seaborn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyod.models.lof import LOF
from pyod.utils.data import generate_data

plt.rcParams['figure.figsize'] = (10, 6)

## 1. Simulação: Dois Departamentos
Imagine que auditamos gastos de dois departamentos:
1.  **TI**: Gasta muito, com variância alta (Cluster denso, valores altos).
2.  **Limpeza**: Gasta pouco, com variância baixa (Cluster muito denso, valores baixos).

Um gasto de 500 reais pode ser normal para TI, mas um absurdo (anomalia) para Limpeza. Um método global poderia não pegar isso.

In [ ]:
# Gerando dois clusters manualmente para ilustrar o problema
np.random.seed(42)

# Cluster 1 (Denso): Limpeza (Valores baixos, pouca variação)
X1 = 0.3 * np.random.randn(100, 2)
X1_outliers = np.random.uniform(low=-4, high=4, size=(10, 2)) # Alguns outliers globais

# Cluster 2 (Esparso): TI (Valores altos, muita variação)
X2 = 3 + 1.5 * np.random.randn(50, 2)

# Juntando tudo
X_train = np.r_[X1, X2]
# Adicionando um outlier local perto do Cluster 1 mas 'longe' para a densidade dele
local_outlier = np.array([[1.5, 1.5]]) 
X_train = np.r_[X_train, local_outlier]

plt.scatter(X_train[:, 0], X_train[:, 1], c='k', s=20)
plt.scatter(local_outlier[:, 0], local_outlier[:, 1], c='red', s=100, label='Outlier Local')
plt.title('Dados com Densidades Diferentes')
plt.legend()
plt.show()

Observe o ponto vermelho. Ele está mais perto da origem (0,0) do que muitos pontos do cluster superior direito (TI). Num método de distância global, ele poderia ser considerado 'normal'. Mas para o cluster da esquerda (Limpeza), ele está muito longe.

In [ ]:
# Aplicando LOF
clf_lof = LOF(n_neighbors=20, contamination=0.1)
clf_lof.fit(X_train)

# Scores
y_pred = clf_lof.labels_
y_scores = clf_lof.decision_scores_

## 2. Visualizando o Resultado

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_pred, cmap='coolwarm', s=30)
plt.title('Detecção via LOF (Vermelho = Outlier)')
plt.xlabel('Dimensão 1')
plt.ylabel('Dimensão 2')
plt.annotate('Outlier Local', xy=(1.5, 1.5), xytext=(3, 0),
             arrowprops=dict(facecolor='black', shrink=0.05),
             )
plt.show()

## Conclusão
O LOF identificou corretamente o outlier local, mesmo que existam pontos 'normais' no outro cluster que estão ainda mais distantes do centro global.

**Aplicação em Auditoria**: Identificar contratos ou pagamentos que fogem do padrão **do fornecedor ou do departamento específico**, mesmo que o valor em si não seja astronomicamente alto para os padrões gerais da empresa.